Important: Each sequence of cells separate by large headers (# in markdown) can be run separately
Each sequence of cells under the same large header should be run sequentially from top to bottom.

# Testing of SVT algorithm with artifical data and given rank

In [1]:
import utils
import svtcpu
import time
import numpy as np

# compute the performance of the prediction
# returns the max error in the given locations, the relative error between the Frobenius norm respect to actual matrix,
# and the rank.
def compute_performance(actual_matrix, predicted_matrix, locations):
    diff = []
    for k in range(len(locations[0])):
        i=locations[0][k]
        j=locations[1][k]
        diff.append(abs(actual_matrix[i,j] - predicted_matrix[i,j])) 
    max_diff = max(diff)

    # compute the relative error between the actual matrix and the resultant matrix
    rel_error = np.linalg.norm(actual_matrix - predicted_matrix, ord='fro') / np.linalg.norm(actual_matrix, ord='fro')

    # compute the rank of the predicted matrix
    u,s,v = np.linalg.svd(predicted_matrix)
    rank = sum(s>0.001)
    return max_diff, rel_error, rank
    

def bulk_test_small_matrices_given_rank(width, height, fixed_entries_num, rank, method = "normal", scale = 1.0, num_trials = 5):
    max_errors = []
    rel_errors = []
    ranks = []
    time_elapsed = []
    for k in range(num_trials):
        # generate a matrix with given rank
        actual_M = utils.generate_matrix_rank(width, height, rank, method = method, scale = scale)
        # generate random locations (of entries) to pass to the sparse matrix
        locations = utils.convert_locations( utils.generate_locations(width, height, fixed_entries_num) )
        # with the locations, create a sparse matrix
        M = utils.filter_locations(actual_M, locations)
        
        # using SVT algorithm, predict the original matrix from the sparse matrix
        time_svt = time.time()
        result = svtcpu.svt_algorithm_auto_params_known_rank(M, locations, rank = rank, log=False)
        time_svt = int(time.time()-time_svt)
        print("time elasped: ", time_svt, " s")
        
        max_diff, rel_error, rank = compute_performance(actual_M, result, locations)
        print("diff on locations:  ",max_diff,"relative error:  ",rel_error, "rank:  ", rank)
        max_errors.append(max_diff)
        rel_errors.append(rel_error)
        ranks.append(rank)
        time_elapsed.append(time_svt)
        
    print("average absolute error: ", np.mean(np.array(max_errors))," average relative error: ", np.mean(np.array(rel_errors)), " average time elapsed: ", np.mean(np.array(time_elapsed)))

## 1000x1000 matrices

In [2]:
print("------------Testing 1000x1000------------")
print("--------Normal, scale=1.0--------")
bulk_test_small_matrices_given_rank(1000, 1000, 120000, 10, method = "normal", scale = 1.0)
bulk_test_small_matrices_given_rank(1000, 1000, 390000, 50, method = "normal", scale = 1.0)
bulk_test_small_matrices_given_rank(1000, 1000, 570000, 100, method = "normal", scale = 1.0)

------------Testing 1000x1000------------
--------Normal, scale=1.0--------
Step size:  10.0     tau:  5000
time elasped:  12  s
diff on locations:   0.07048942238063322 relative error:   0.00164222095417725 rank:   10
Step size:  10.0     tau:  5000
time elasped:  12  s
diff on locations:   0.03285203443459528 relative error:   0.0015855425863229484 rank:   10
Step size:  10.0     tau:  5000
time elasped:  13  s
diff on locations:   0.06343395797089268 relative error:   0.001641685212990671 rank:   10
Step size:  10.0     tau:  5000
time elasped:  12  s
diff on locations:   0.04533275461455588 relative error:   0.0016406647538154938 rank:   10
Step size:  10.0     tau:  5000
time elasped:  12  s
diff on locations:   0.04934980754568219 relative error:   0.0015896341269456462 rank:   10
average absolute error:  0.05229159538927185  average relative error:  0.001619949526850402  average time elapsed:  12.2
Step size:  3.076923076923077     tau:  5000
time elasped:  50  s
diff on locatio

## 5000x5000 matrices

In [2]:
print("------------Testing 5000x5000------------")
bulk_test_small_matrices_given_rank(5000, 5000, 600000, 10, method = "normal", scale = 1.0)
bulk_test_small_matrices_given_rank(5000, 5000, 600000, 10, method = "uniform", scale = 1.0)

------------Testing 5000x5000------------
Step size:  50.0     tau:  25000
time elasped:  385  s
diff on locations:   0.06461660686870552 relative error:   0.001706489698112507 rank:   10
Step size:  50.0     tau:  25000
time elasped:  400  s
diff on locations:   0.07635663327220366 relative error:   0.0016323038530537378 rank:   10
Step size:  50.0     tau:  25000
time elasped:  376  s
diff on locations:   0.06389376401175095 relative error:   0.0016666222630925811 rank:   10
Step size:  50.0     tau:  25000
time elasped:  394  s
diff on locations:   0.08585635191938223 relative error:   0.001669331809080294 rank:   10
Step size:  50.0     tau:  25000
time elasped:  373  s
diff on locations:   0.06356849205868009 relative error:   0.0016406124908615925 rank:   10
average absolute error:  0.07085836962614449  average relative error:  0.0016630720228401424  average time elapsed:  385.6
Step size:  50.0     tau:  25000
time elasped:  1470  s
diff on locations:   0.027250450517046332 rela

# Comparison of SVT algorithm with gradient descent

In [3]:
import utils
import svtcpu
import time
import numpy as np
import descent

# same function as above
def compute_performance(actual_matrix, predicted_matrix, locations):
    diff = []
    for k in range(len(locations[0])):
        i=locations[0][k]
        j=locations[1][k]
        diff.append(abs(actual_matrix[i,j] - predicted_matrix[i,j])) 
    max_diff = max(diff)

    # compute the relative error between the actual matrix and the resultant matrix
    rel_error = np.linalg.norm(actual_matrix - predicted_matrix, ord='fro') / np.linalg.norm(actual_matrix, ord='fro')

    # compute the rank of the resultant matrix
    u,s,v = np.linalg.svd(predicted_matrix)
    rank = sum(s>0.001)
    return max_diff, rel_error, rank

# comparison of SVT with gradient descent
def bulk_compare_small_matrices_given_rank(width, height, fixed_entries_num, rank, method = "normal", scale = 1.0, num_trials = 5):
    max_errors_SVT = []
    rel_errors_SVT = []
    ranks_SVT = []
    time_elapsed_SVT = []
    
    max_errors_descent = []
    rel_errors_descent = []
    ranks_descent = []
    time_elapsed_descent = []
    for k in range(num_trials):
        # generate a matrix with given rank
        actual_M = utils.generate_matrix_rank(width, height, rank, method = method, scale = scale)
        # generate random locations (of entries) to pass to the sparse matrix
        locations = utils.convert_locations( utils.generate_locations(width, height, fixed_entries_num) )
        # with the locations, create a sparse matrix
        M = utils.filter_locations(actual_M, locations)
        
        # using SVT algorithm, predict the original matrix from the sparse matrix
        time_svt = time.time()
        result = svtcpu.svt_algorithm_auto_params_known_rank(M, locations, rank = rank, log=False, tolerance = 0.01)
        time_svt = int(time.time()-time_svt)
        print("time elasped: ", time_svt, " s")
        
        # compute the absolute difference of values in entries in the locations,
        # for the actual matrix and the resultant matrix
        max_diff, rel_error, rank = compute_performance(actual_M, result, locations)
        
        print("diff on locations:  ",max_diff,"relative error:  ",rel_error, "rank:  ", rank, "     (SVT)")
        max_errors_SVT.append(max_diff)
        rel_errors_SVT.append(rel_error)
        ranks_SVT.append(rank)
        time_elapsed_SVT.append(time_svt)
        
        
        # using gradient descent, predict the original matrix from the sparse matrix
        time_descent = time.time()
        result = descent.gradient_descent_completion(M, locations, rank = rank, log=False, tolerance = 0.01)
        time_descent = int(time.time()-time_descent)
        print("time elasped: ", time_descent, " s")
        
        # compute the absolute difference of values in entries in the locations,
        # for the actual matrix and the resultant matrix
        max_diff, rel_error, rank = compute_performance(actual_M, result, locations)
        
        print("diff on locations:  ",max_diff,"relative error:  ",rel_error, "rank:  ", rank, "     (descent)")
        max_errors_descent.append(max_diff)
        rel_errors_descent.append(rel_error)
        ranks_descent.append(rank)
        time_elapsed_descent.append(time_descent)
    print("average absolute error: ", np.mean(np.array(max_errors_SVT))," average relative error: ", np.mean(np.array(rel_errors_SVT)), " average time elapsed: ", np.mean(np.array(time_elapsed_SVT)), "    (SVT)")
    print("average absolute error: ", np.mean(np.array(max_errors_descent))," average relative error: ", np.mean(np.array(rel_errors_descent)), " average time elapsed: ", np.mean(np.array(time_elapsed_descent)), "    (descent)")

## 5000x5000 matrices

In [4]:
print("------------Testing 5000x5000------------")
bulk_compare_small_matrices_given_rank(5000, 5000, 600000, 10, method = "normal", scale = 1.0, num_trials = 1)

------------Testing 5000x5000------------
Step size:  50.0     tau:  25000
time elasped:  199  s
diff on locations:   0.3420094642757965 relative error:   0.016004591936981893 rank:   10      (SVT)
torch.Size([5000, 10])
torch.Size([5000, 10])
time elasped:  1597  s
diff on locations:   0.7582143193855684 relative error:   0.016623951228249195 rank:   10      (descent)
average absolute error:  0.3420094642757965  average relative error:  0.016004591936981893  average time elapsed:  199.0     (SVT)
average absolute error:  0.7582143193855684  average relative error:  0.016623951228249195  average time elapsed:  1597.0     (descent)
